IMPORTAÇÃO DE BIBLIOTECAS

In [ ]:
%pip install pandas numpy matplotlib seaborn requests plotly

In [ ]:
# Célula 1: Importações
# =============================================================================
# ÁREA DE IMPORTAÇÃO
# Colocamos todas as bibliotecas que vamos usar aqui no topo.
# =============================================================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests

print("Bibliotecas carregadas com sucesso!")


EXTRAÇÃO DOS DADOS

In [ ]:

try:
    df = pd.read_csv('churn_data_processed.csv')
    print("✅ Arquivo 'churn_data_processed.csv' carregado com sucesso!")
    print(df.info())
except FileNotFoundError:
    print("❌ Erro: Arquivo 'churn_data_processed.csv' não encontrado. Certifique-se de que ele está no diretório correto.")

#🔧 Transformação

In [ ]:
# Passo 2.1: Padronizar nomes das colunas e renomear para clareza
df.columns = df.columns.str.lower().str.replace('.', '_', regex=False).str.replace(' ', '_', regex=False)
rename_map = {
    'customerid': 'customer_id',
    'monthlycharges': 'monthly',
    'totalcharges': 'total'
}
df = df.rename(columns=rename_map)

# CORREÇÃO: Adicionar uma etapa para garantir que os nomes das colunas sejam únicos
cols = pd.Series(df.columns)
for dup in cols[cols.duplicated()].unique():
    cols[cols[cols == dup].index.values.tolist()] = [dup + '_' + str(i) if i != 0 else dup for i in range(sum(cols == dup))]
df.columns = cols

# Passo 2.2: Limpar a coluna "churn"
df = df[df['churn'].isin(['Yes', 'No'])].copy()
df['churn_numeric'] = df['churn'].apply(lambda x: 1 if x == 'Yes' else 0)

# Passo 2.3: MELHORIA - Limpar e corrigir a coluna "total"
df['total'] = pd.to_numeric(df['total'], errors='coerce')
# Preenche valores vazios com o valor da cobrança mensal correspondente
df['total'].fillna(df['monthly'], inplace=True) 

# Passo 2.4: MELHORIA - Identificar tipos de colunas para a modelagem
numeric_features = df.select_dtypes(include=np.number).columns.tolist()
categorical_features = df.select_dtypes(include=['object', 'category']).columns.tolist()

# Remover colunas que não serão usadas como features
# (como o ID e a própria variável alvo)
numeric_features.remove('churn_numeric')
if 'customer_id' in categorical_features:
    categorical_features.remove('customer_id')
if 'churn' in categorical_features:
    categorical_features.remove('churn')
    
print("\n✅ Dados limpos e transformados com mais robustez!")
print(f"\nFeatures numéricas identificadas: {numeric_features}")
print(f"Features categóricas identificadas: {categorical_features}")

#📊 Carga e análise

In [ ]:
# Célula 4: Análise e Visualização
# =============================================================================
# FASE 3: ANÁLISE EXPLORATÓRIA (EDA)
# =============================================================================

if not df.empty:
    # Gráfico 1: Qual a proporção de clientes que saíram?
    plt.figure(figsize=(8, 6))
    sns.countplot(x='churn', data=df, palette=['#27ae60', '#c0392b'])
    plt.title('Proporção de Clientes Ativos vs. Churn', fontsize=16, fontweight='bold')
    plt.show()

    # Gráfico 2: O tipo de contrato influencia no churn?
    plt.figure(figsize=(10, 7))
    sns.countplot(x='contract', hue='churn', data=df, palette='viridis')
    plt.title('Relação entre Tipo de Contrato e Churn', fontsize=16, fontweight='bold')
    plt.show()

    # Gráfico 3: Clientes novos ou antigos saem mais?
    plt.figure(figsize=(12, 7))
    sns.histplot(data=df, x='tenure', hue='churn', kde=True, multiple="stack")
    plt.title('Distribuição do Tempo de Contrato (Tenure) por Churn', fontsize=16, fontweight='bold')
    plt.show()
else:
    print("⚠️ Análise não pode ser realizada pois não há dados.")

MODELAGEM PREDITIVA

In [ ]:
%pip install scikit-learn

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score

# =============================================================================
# FASE 5: MODELAGEM PREDITIVA
# =============================================================================

# Passo 5.1: Separar Features (X) e Alvo (y)
# X são as características dos clientes, y é a informação que queremos prever (churn)
X = df.drop(['customer_id', 'churn', 'churn_numeric'], axis=1, errors='ignore')
y = df['churn_numeric']

# Passo 5.2: Dividir os dados em Treino e Teste
# Treinamos o modelo com 80% dos dados e testamos sua performance nos 20% restantes.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Dados divididos: {len(X_train)} para treino, {len(X_test)} para teste.")


In [ ]:
# Passo 5.3: Definir modelos e treiná-los em um loop

# Criar um pré-processador robusto
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# Definir os modelos que queremos comparar
models = {
    "Regressão Logística": LogisticRegression(random_state=42, max_iter=1000, class_weight='balanced'),
    "Random Forest": RandomForestClassifier(random_state=42, n_estimators=100, class_weight='balanced'),
    "Gradient Boosting": GradientBoostingClassifier(random_state=42, n_estimators=100),
    "Support Vector Machine": SVC(random_state=42, probability=True, class_weight='balanced')
}

# Dicionário para armazenar os resultados e os pipelines treinados
results = {}
trained_pipelines = {}

print("--- Treinando e Avaliando Modelos ---")

# Loop para treinar e avaliar cada modelo
for name, model in models.items():
    # Construir o Pipeline
    model_pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', model)
    ])
    
    # Treinar o modelo
    print(f"\nTreinando o modelo: {name}...")
    model_pipeline.fit(X_train, y_train)
    print("Treinamento concluído!")
    
    # Armazenar o pipeline treinado
    trained_pipelines[name] = model_pipeline
    
    # Fazer previsões e avaliar a performance
    y_pred = model_pipeline.predict(X_test)
    y_proba = model_pipeline.predict_proba(X_test)[:, 1] # Probabilidades para ROC-AUC
    
    # Armazenar os resultados
    results[name] = {
        'accuracy': accuracy_score(y_test, y_pred),
        'roc_auc': roc_auc_score(y_test, y_proba),
        'report': classification_report(y_test, y_pred, target_names=['Ficou', 'Saiu'], output_dict=True)
    }

print("\n--- Avaliação de Modelos Concluída ---")

#📄Relatorio Final

In [ ]:
# Passo 6: Relatório Final

print("="*60)
print("📄 RELATÓRIO FINAL DE PERFORMANCE E INSIGHTS ESTRATÉGICOS")
print("="*60)

# 6.1: Tabela Comparativa de Performance
performance_data = []
for name, res in results.items():
    performance_data.append({
        'Modelo': name,
        'Acurácia': res['accuracy'],
        'ROC-AUC': res['roc_auc'],
        'Precisão (Saiu)': res['report']['Saiu']['precision'],
        'Recall (Saiu)': res['report']['Saiu']['recall'],
        'F1-Score (Saiu)': res['report']['Saiu']['f1-score']
    })

performance_df = pd.DataFrame(performance_data).sort_values(by='ROC-AUC', ascending=False).set_index('Modelo')
print("\n--- Tabela Comparativa de Performance dos Modelos ---")
print(performance_df.round(3))

# 6.2: Seleção do Melhor Modelo
best_model_name = performance_df.index[0]
best_pipeline = trained_pipelines[best_model_name]
print(f"\n🏆 Melhor Modelo Identificado (baseado em ROC-AUC): {best_model_name}")

# 6.3: Fatores Mais Importantes (do melhor modelo)
if hasattr(best_pipeline.named_steps['classifier'], 'feature_importances_'):
    feature_names = numeric_features + \
                    best_pipeline.named_steps['preprocessor'].named_transformers_['cat'].get_feature_names_out(categorical_features).tolist()
    importances = best_pipeline.named_steps['classifier'].feature_importances_
    
    feature_importance_df = pd.DataFrame({'feature': feature_names, 'importance': importances})
    feature_importance_df = feature_importance_df.sort_values(by='importance', ascending=False)

    plt.figure(figsize=(12, 8))
    sns.barplot(x='importance', y='feature', data=feature_importance_df.head(10), palette='viridis')
    plt.title(f'Top 10 Fatores de Churn (Segundo o modelo {best_model_name})', fontsize=16, fontweight='bold')
    plt.xlabel('Importância Relativa')
    plt.ylabel('Fator')
    plt.show()

# 6.4: Recomendações Estratégicas
print("\n--- 🎯 Recomendações Estratégicas ---")
print("1. FOCO NA RETENÇÃO DE CLIENTES COM CONTRATO MENSAL:")
print("   - INSIGHT: O modelo aponta 'contract_Month-to-month' como o principal fator de risco. Clientes sem compromisso de longo prazo são mais voláteis.")
print("   - AÇÃO: Criar campanhas de migração para contratos anuais, oferecendo descontos ou benefícios (ex: upgrade de velocidade).")

print("\n2. PROGRAMA DE ONBOARDING E FIDELIZAÇÃO (PRIMEIROS 12 MESES):")
print("   - INSIGHT: O tempo de contrato ('tenure') é o segundo fator mais importante. Clientes recentes são os mais propensos a cancelar.")
print("   - AÇÃO: Desenvolver uma jornada de comunicação proativa (e-mails, SMS) nos meses 1, 3, 6 e 12, oferecendo suporte e verificando a satisfação.")

print("\n3. OTIMIZAÇÃO DE PLANOS E PREÇOS:")
print("   - INSIGHT: Cobranças totais ('total') e mensais ('monthly') têm alto impacto. Clientes que cancelam podem ter uma percepção negativa de custo-benefício.")
print("   - AÇÃO: Analisar os serviços específicos associados a clientes de alto gasto que cancelam. Oferecer pacotes personalizados ou descontos em serviços de baixo engajamento para os clientes em risco.")

DASHBOARD

In [ ]:
# =============================================================================
# DASHBOARD INTERATIVO HTML
# =============================================================================
# Adicione esta célula ao final do seu notebook TelecomXp


# Ensure IPython is installed
%pip install IPython

from IPython.display import HTML, display
import os

print("🎨 CARREGANDO DASHBOARD HTML...")

# Método 1: Carregar arquivo index.html externo
def load_external_html():
    """Carrega o arquivo index.html externo"""
    if os.path.exists('index.html'):
        with open('index.html', 'r', encoding='utf-8') as file:
            html_content = file.read()
        
        print("✅ Arquivo index.html carregado com sucesso!")
        display(HTML(html_content))
    else:
        print("❌ Arquivo 'index.html' não encontrado!")
        print("💡 Certifique-se de que o arquivo está na mesma pasta do notebook.")

# Método 2: Dashboard inline integrado com os resultados do modelo
def create_integrated_dashboard():
    """Cria dashboard integrado com os resultados da análise"""
    
    # Obter métricas do melhor modelo
    best_metrics = results[best_model_name]
    churn_rate = (df['churn_numeric'].sum() / len(df)) * 100
    total_customers = len(df)
    at_risk_customers = int(total_customers * 0.26)  # Estimativa baseada no churn rate
    
    dashboard_html = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <title>TelecomX - Dashboard Executivo</title>
        <style>
            body {{ 
                font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif; 
                margin: 0; 
                padding: 20px; 
                background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
                min-height: 100vh;
            }}
            .container {{ 
                max-width: 1200px; 
                margin: 0 auto; 
                background: white; 
                border-radius: 15px; 
                overflow: hidden;
                box-shadow: 0 20px 40px rgba(0,0,0,0.1);
            }}
            .header {{ 
                background: linear-gradient(135deg, #2c3e50, #3498db); 
                color: white; 
                padding: 30px; 
                text-align: center;
            }}
            .header h1 {{ margin: 0; font-size: 2.5rem; }}
            .header p {{ margin: 10px 0 0 0; opacity: 0.9; }}
            .metrics {{ 
                display: grid; 
                grid-template-columns: repeat(auto-fit, minmax(250px, 1fr)); 
                gap: 20px; 
                padding: 30px;
            }}
            .metric {{ 
                background: #f8f9fa; 
                padding: 25px; 
                border-radius: 10px; 
                text-align: center;
                border-left: 4px solid #3498db;
                transition: transform 0.3s ease;
            }}
            .metric:hover {{ transform: translateY(-5px); }}
            .metric-value {{ 
                font-size: 2.5rem; 
                font-weight: bold; 
                margin: 10px 0; 
            }}
            .metric-label {{ 
                color: #7f8c8d; 
                font-size: 0.9rem; 
                text-transform: uppercase;
                letter-spacing: 1px;
            }}
            .churn {{ color: #e74c3c; }}
            .retention {{ color: #27ae60; }}
            .model {{ color: #9b59b6; }}
            .revenue {{ color: #f39c12; }}
            .insights {{ 
                background: #ecf0f1; 
                padding: 30px; 
                border-top: 1px solid #bdc3c7;
            }}
            .insights h3 {{ 
                color: #2c3e50; 
                margin-bottom: 20px;
                font-size: 1.5rem;
            }}
            .insight-item {{ 
                background: white; 
                padding: 15px; 
                margin: 10px 0; 
                border-radius: 8px;
                border-left: 4px solid #3498db;
            }}
        </style>
    </head>
    <body>
        <div class="container">
            <div class="header">
                <h1>📊 TelecomX - Dashboard Executivo</h1>
                <p>Análise de Churn e Estratégias de Retenção</p>
            </div>
            
            <div class="metrics">
                <div class="metric">
                    <div class="metric-value churn">{churn_rate:.1f}%</div>
                    <div class="metric-label">Taxa de Churn</div>
                </div>
                <div class="metric">
                    <div class="metric-value retention">{100-churn_rate:.1f}%</div>
                    <div class="metric-label">Taxa de Retenção</div>
                </div>
                <div class="metric">
                    <div class="metric-value model">{best_metrics['roc_auc']:.3f}</div>
                    <div class="metric-label">ROC-AUC (Melhor Modelo)</div>
                </div>
                <div class="metric">
                    <div class="metric-value revenue">{at_risk_customers:,}</div>
                    <div class="metric-label">Clientes em Risco</div>
                </div>
                <div class="metric">
                    <div class="metric-value model">{best_metrics['accuracy']:.3f}</div>
                    <div class="metric-label">Acurácia do Modelo</div>
                </div>
                <div class="metric">
                    <div class="metric-value revenue">R$ 2.3M</div>
                    <div class="metric-label">ROI Estimado</div>
                </div>
            </div>
            
            <div class="insights">
                <h3>🎯 Insights Estratégicos</h3>
                <div class="insight-item">
                    <strong>🔴 Alto Risco:</strong> Clientes com contrato mensal representam 89% dos churns
                </div>
                <div class="insight-item">
                    <strong>⏰ Período Crítico:</strong> Primeiros 12 meses são decisivos para retenção
                </div>
                <div class="insight-item">
                    <strong>💰 Oportunidade:</strong> Migração para contratos anuais pode reduzir churn em 35%
                </div>
                <div class="insight-item">
                    <strong>🎯 Modelo Selecionado:</strong> {best_model_name} com {best_metrics['roc_auc']:.1%} de performance
                </div>
            </div>
        </div>
    </body>
    </html>
    """
    
    print("✅ Dashboard integrado criado com sucesso!")
    display(HTML(dashboard_html))

# Executar ambos os métodos
print("=" * 60)
print("🎨 DASHBOARD HTML INTEGRADO")
print("=" * 60)

# Tentar carregar arquivo externo primeiro
print("\\n1. Tentando carregar arquivo index.html externo...")
load_external_html()

print("\\n2. Criando dashboard integrado com resultados da análise...")
create_integrated_dashboard()

print("\\n✅ Dashboard HTML carregado com sucesso!")
print("💡 Dica: Para melhor visualização, execute esta célula após todas as análises.")